In [ ]:
import pandas as pd
from plotnine import (
    aes,
    element_blank,
    facet_wrap,
    geom_line,
    geom_ribbon,
    ggplot,
    labs,
    scale_color_manual,
    scale_x_date,
    theme,
    theme_bw,
)

from config import ROOT
from src.plot_functions import get_sundays, prepare_plot_data
from src.realtime_utils import load_rt, load_target_series

In [ ]:
start = "2023-11-16"
end = "2024-09-12"

In [ ]:
files = sorted((ROOT / "nowcasts" / "simple_nowcast").glob("*.csv"))

dfs = []
for file in files:
    df_temp = pd.read_csv(file, parse_dates=["forecast_date", "target_end_date"])
    dfs.append(df_temp)
df = pd.concat(dfs)

df = df[df.forecast_date.between(start, end)]
df = df[df.type == "quantile"]

target_sari = load_target_series("sari")

df_plot = prepare_plot_data(df, target_sari)

In [ ]:
def load_frozen_values(horizon):
    rt = load_rt(preprocessed=False)
    rt = rt.dropna()
    rt["value"] = rt.loc[:, "value_0w" : f"value_{-1 * horizon}w"].sum(axis=1)
    return rt[[c for c in rt.columns if "value_" not in c]]

In [ ]:
df_frozen = pd.DataFrame()
for h in [0, -1, -2, -3]:
    df_temp = load_frozen_values(h)
    df_temp["horizon"] = h
    df_frozen = pd.concat([df_frozen, df_temp])

df_temp = df_plot[(df_plot.location == "DE") & (df_plot.age_group == "00+")]
frozen_temp = df_frozen[(df_frozen.location == "DE") & (df_frozen.age_group == "00+")]

y_temp = df_temp[df_temp.type == "truth"]
y_temp = pd.concat([y_temp.assign(horizon=h) for h in [0, -1, -2, -3]], ignore_index=True)

df_temp = df_temp[df_temp.type != "truth"]
y_temp = y_temp[y_temp.target_end_date.between(df_temp.target_end_date.min(), df_temp.target_end_date.max())]
frozen_temp = frozen_temp[frozen_temp.date.between(df_temp.target_end_date.min(), df_temp.target_end_date.max())]

# Get all Sundays within the range of the dataset
sundays = get_sundays(df_temp["target_end_date"].min(), df_temp["target_end_date"].max())

In [ ]:
p = (
    ggplot(df_temp, aes(x="target_end_date"))
    + facet_wrap(
        "horizon", ncol=2, scales="free_y", labeller=lambda x: "Horizon: " + str(x)
    )  # + (' weeks' if x != '-1' else ' week')) +
    + geom_ribbon(aes(ymin="quantile_0.05", ymax="quantile_0.95"), fill="blue", alpha=0.3)
    + geom_line(aes(y="quantile_0.5"), color="blue")
    + geom_line(frozen_temp, aes(x="date", y="value", color="'frozen'"))
    + geom_line(y_temp, aes(x="target_end_date", y="quantile_0.5", color="'final'"))
    + scale_color_manual(
        name="Truth",
        values={"final": "darkred", "frozen": "gray"},
        labels={"frozen": "At time of nowcast", "final": "Final"},
    )
    + scale_x_date(
        date_breaks="4 months",
        # breaks=[pd.Timestamp('2024-01-01'), pd.Timestamp('2024-03-01'), pd.Timestamp('2024-05-01')],
        minor_breaks=sundays,  # Add minor ticks at every Sunday
    )
    + labs(x="", y="", title="National level")
    + theme_bw()
    + theme(
        panel_grid_major_x=element_blank(),
        panel_grid_major_y=element_blank(),
        panel_grid_minor_y=element_blank(),
        panel_grid_minor=element_blank(),
        legend_key=element_blank(),
    )
)

p.show()

In [ ]:
p.save(
    ROOT / "figures" / "FigureS5.pdf",
    width=190.5,
    height=90,
    units="mm",
    dpi=600,
    verbose=False,
)